In [ ]:
globals().clear
import time
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
from datetime import datetime

In [ ]:
# Install Libraries

# mount Google Drive
from google.colab import drive
drive.mount('/content/colabIntern')
#%cd "/content/colabIntern/MyDrive/2023/Langara/Capstone Project/Collab"
%cd "/content/colabIntern/MyDrive/Capstone/Capstone Project/Collab"
!ls | wc -l

Mounted at /content/colabIntern
/content/colabIntern/.shortcut-targets-by-id/1lwwFhOdnZo3Bx0mzGW8l1qPio2d8s1TV/Langara/Capstone Project/Collab
6


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler

import tensorflow       as tf
import keras.optimizers as op
from keras.callbacks import EarlyStopping

from tensorflow                     import keras
from tensorflow.keras.models        import Sequential
from tensorflow.keras.layers        import Dense
from tensorflow.keras.layers        import Dropout
from tensorflow.keras.optimizers    import Adam

In [ ]:
# Load Dataset
df = pd.read_csv('df.csv')

In [ ]:
# list of tickers for stocks in our data set. Sort the ticker list since our df will be alphabetically arranged.
tickers = ["AAPL", 'XOM', 'IBM', 'KO', 'CVX', 'BA', 'PFE', 'MSFT', 'T', 'WMT',
       'F', 'NFLX', 'JPM', 'MCD', 'GE', 'NVDA', 'JNJ', 'BAC', 'C', 'AMZN',
       'INTC', 'CSCO', 'TSLA', 'GOOGL', 'AMD', 'BABA', 'VZ', 'DIS',
       'META', 'PLTR']
tickers.sort()

In [ ]:
# For this specific trial drop PLTR since data is not complete
df = df[df.stock_ID != "PLTR"]

In [ ]:
#df1=df.iloc[::,:74]
#df1


In [ ]:
df['DATETIME']= pd.to_datetime(df['DATETIME'], format='%m/%d/%Y %H:%M')

In [ ]:
# Sort by time so that the first 29 rows occupy the first time value for all the stocks.
# Fill in 0's for missing values for now.
df = df.set_index('DATETIME')
df = df.fillna(0)

In [ ]:
# Select the length of the df ; For this file we need 16 months
# We use the first 12 months to make the first prediction, then shift window
# Then we will repeat this for the next 4 months (Hence total is 16 months needed)
df = df.sort_index().loc['2021-09-01':'2022-12-30']

In [ ]:
#Set the DATETIME for fecha
DATETIME = df.index.values

In [ ]:
# Sort dataframe such that it is both in sequential order, and also in alphabetical order for each day
#(i.e first entry for each time entry should be AAPL, and last should be XOM).
df.sort_values(["DATETIME", "stock_ID"], inplace=True)

In [ ]:
df.head()

,stock_ID,Close,Volume_PP,US2Y_PP,US5Y_PP,US10Y_PP,DJ_PP,NQ_PP,SP_PP,Gold_PP,...,Sector_Financial Services,Sector_Healthcare,Sector_Industrial,Sector_Technology,P/E Ratio ttm,P/B Ratio mrq,D/E Ratio mrq,Free Cash Flow ttm,PEG Ratio 5 year expected,ROE ttm
DATETIME,,,,,,,,,,,,,,,,,,,,,
2021-09-01 09:30:00,AAPL,154.41,5792047.0,0.04,0.78,1.3,35309.01,15329.78,4525.8,1813.1,...,0,0,0,1,27.96,45.93,195.87,84.73,2.78,147.94
2021-09-01 09:30:00,AMD,110.90,483413.0,0.04,0.78,1.3,35309.01,15329.78,4525.8,1813.1,...,0,0,0,1,110.08,2.72,5.40,3.79,1.81,4.24
2021-09-01 09:30:00,AMZN,175.28,3941440.0,0.04,0.78,1.3,35309.01,15329.78,4525.8,1813.1,...,0,0,0,0,0.00,7.16,116.36,7.62,2.30,-1.92
2021-09-01 09:30:00,BA,220.67,126161.0,0.04,0.78,1.3,35309.01,15329.78,4525.8,1813.1,...,0,0,1,0,0.00,0.00,0.00,5.08,6.53,0.00
2021-09-01 09:30:00,BABA,171.83,866535.0,0.04,0.78,1.3,35309.01,15329.78,4525.8,1813.1,...,0,0,0,0,56.28,1.87,17.37,41.83,2.10,2.28


In [ ]:
df=df.iloc[:,:74]
df.head()

,stock_ID,Close,Volume_PP,US2Y_PP,US5Y_PP,US10Y_PP,DJ_PP,NQ_PP,SP_PP,Gold_PP,...,DAY_22,DAY_23,DAY_24,DAY_25,DAY_26,DAY_27,DAY_28,DAY_29,DAY_30,DAY_31
DATETIME,,,,,,,,,,,,,,,,,,,,,
2021-09-01 09:30:00,AAPL,154.41,5792047.0,0.04,0.78,1.3,35309.01,15329.78,4525.8,1813.1,...,0,0,0,0,0,0,0,0,0,0
2021-09-01 09:30:00,AMD,110.90,483413.0,0.04,0.78,1.3,35309.01,15329.78,4525.8,1813.1,...,0,0,0,0,0,0,0,0,0,0
2021-09-01 09:30:00,AMZN,175.28,3941440.0,0.04,0.78,1.3,35309.01,15329.78,4525.8,1813.1,...,0,0,0,0,0,0,0,0,0,0
2021-09-01 09:30:00,BA,220.67,126161.0,0.04,0.78,1.3,35309.01,15329.78,4525.8,1813.1,...,0,0,0,0,0,0,0,0,0,0
2021-09-01 09:30:00,BABA,171.83,866535.0,0.04,0.78,1.3,35309.01,15329.78,4525.8,1813.1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
num_companies = 29  # 29 companies in our dataset.
step_rows  = 24 * num_companies  # 24 time periods per day per stock
total_rows = len(df['2021-09-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(df['2021-09-01':'2022-08-31'])  # Define length of training window


scale_X = MinMaxScaler()
model = Sequential()
model.add(Dense(100, activation = 'relu', input_dim = df.iloc[:,1:-1].shape[1]))  # initiating with 100 neurons
model.add(Dense(100, activation = 'relu', input_dim = df.iloc[:,1:-1].shape[1]))
model.add(Dropout(0.25))                                                          # adding dropout to avoid overfitting
model.add(Dense(1))                                                               # output layer

opt = Adam(amsgrad = True, lr = 0.001, beta_1 = 0.79, beta_2 = 0.999)             # using Adam optimizer, at a learning rate of 0.001
model.compile(loss = 'mse', optimizer = opt)                                      # compiling model
#es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = df.iloc[i:train_rows + i, 1:]
    test   = df.iloc[train_rows + i:train_rows + i + step_rows, 1:]
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]



    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model.fit(X_train, y_train, epochs=20, batch_size=256) #, callbacks=[es]) #, epochs=8, batch_size=256
    y_hat = model.predict(X_test, verbose=False)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    result = pd.concat([result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Epoch 1/20
669/669 [==============================] - 4s 3ms/step - loss: 10976.8721
Epoch 2/20
669/669 [==============================] - 4s 6ms/step - loss: 9517.2139
Epoch 3/20
669/669 [==============================] - 2s 4ms/step - loss: 9465.6680
Epoch 4/20
669/669 [==============================] - 2s 3ms/step - loss: 9456.7314
Epoch 5/20
669/669 [==============================] - 2s 3ms/step - loss: 9435.8721
Epoch 6/20
669/669 [==============================] - 2s 3ms/step - loss: 9435.0713
Epoch 7/20
669/669 [==============================] - 2s 4ms/step - loss: 9420.9834
Epoch 8/20
669/669 [==============================] - 3s 5ms/step - loss: 9421.5625
Epoch 9/20
669/669 [==============================] - 2s 3ms/step - loss: 9407.0684
Epoch 10/20
669/669 [==============================] - 2s 3ms/step - loss: 9408.3555
Epoch 11/20
669/669 [==============================] - 2s 3ms/step - loss: 9407.9473
Epoch 12/20
669/669 [==============================] - 2s 3ms/step - loss

In [ ]:
metrics_df = pd.DataFrame(columns = ['stock_ID', "RMSE", "MAPE", "MPE", "MTT"])

In [ ]:
result[25::29]

,DATETIME,ACTUAL,PREDICTED,DIFFERENCE,TRAIN_DURATION
25,2022-09-01 09:30:00,274.66,125.033150,149.626850,83.067599
54,2022-09-01 09:45:00,273.62,118.098358,155.521642,83.067599
83,2022-09-01 10:00:00,271.36,123.935112,147.424888,83.067599
112,2022-09-01 10:15:00,273.14,121.526611,151.613389,83.067599
141,2022-09-01 10:30:00,269.54,122.368805,147.171195,83.067599
...,...,...,...,...,...
56256,2022-12-30 14:15:00,121.46,114.909180,6.550820,82.571046
56285,2022-12-30 14:30:00,121.30,109.462158,11.837842,82.571046
56314,2022-12-30 14:45:00,121.51,109.282593,12.227407,82.571046
56343,2022-12-30 15:00:00,122.42,115.503510,6.916490,82.571046


In [ ]:
def mean_positive_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    error = np.mean(np.maximum((y_pred - y_true),0))
    return error

In [ ]:
new_rows = []
for i in range(len(tickers)):
    stock_result = result[i::29]
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
    # Calculate MAPE %
    mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
    # Calculate MPE %
    mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
    # Calculate MTT in seconds
    mtt = np.mean(stock_result['TRAIN_DURATION'])
    new_row = pd.Series([tickers[i],rmse, mape, mpe, mtt], index=metrics_df.columns)
    new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)

In [ ]:
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,AAPL,35.939075,22.310967,0.418919,67.386235
1,AMD,49.868856,70.027800,46.158649,67.386235
2,AMZN,22.910062,18.409616,12.797788,67.386235
3,BA,37.881010,19.626268,4.381012,67.386235
4,BABA,32.813362,35.522088,27.063140,67.386235
5,BAC,72.746088,211.080427,71.590074,67.386235
6,C,62.943311,132.919779,61.039878,67.386235
7,CSCO,66.069199,142.444831,64.048470,67.386235
8,CVX,44.357569,23.079987,1.172692,67.386235
9,DIS,24.264740,19.589450,17.688149,67.386235


In [ ]:
metrics_df.drop("stock_ID",axis=1)

,RMSE,MAPE,MPE,MTT
0,35.939075,22.310967,0.418919,67.386235
1,49.868856,70.027800,46.158649,67.386235
2,22.910062,18.409616,12.797788,67.386235
3,37.881010,19.626268,4.381012,67.386235
4,32.813362,35.522088,27.063140,67.386235
5,72.746088,211.080427,71.590074,67.386235
6,62.943311,132.919779,61.039878,67.386235
7,66.069199,142.444831,64.048470,67.386235
8,44.357569,23.079987,1.172692,67.386235
9,24.264740,19.589450,17.688149,67.386235


In [ ]:
avg_values=metrics_df.mean()

<ipython-input-93-bfd6476c2058>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  avg_values=metrics_df.mean()


In [ ]:
avg_df = pd.DataFrame(avg_values).T
avg_df

,RMSE,MAPE,MPE,MTT
0,54.549918,83.114028,25.586097,67.385966
